### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Adrenocortical_Cancer/GSE143383'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # initialize and define the functions separately

# Determine if gene expression data is available
is_gene_available = True

# Sample Characteristics Dictionary (derived from the output of STEP 1)
sample_dict = {
    0: ['gender: M', 'gender: F', 'gender: unknown'],
    1: ['Adrenocortical_Cancer: yes', 'Adrenocortical_Cancer: no'],
    2: ['age: 30', 'age: 45', 'age: unknown']
}

# Determine the availability of variables
# gender information is available at key 0
gender_row = 0

# Adrenocortical_Cancer information is available at key 1
trait_row = 1

# Age information is available at key 2
age_row = 2

# Define conversion functions
def convert_gender(value):
    value = value.split(':')[-1].strip().lower()
    if value == 'm':
        return 1
    elif value == 'f':
        return 0
    else:
        return None

def convert_trait(value):
    value = value.split(':')[-1].strip().lower()
    if value == 'yes':
        return 1
    elif value == 'no':
        return 0
    else:
        return None

def convert_age(value):
    value = value.split(':')[-1].strip().lower()
    try:
        return int(value)
    except (ValueError, TypeError):
        return None

# Save cohort information
save_cohort_info('GSE143383', './preprocessed/Adrenocortical_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    if clinical_data.empty:
        print("Error: clinical_data is empty")
    else:
        print("Clinical Data Sample:", clinical_data.head())
        selected_clinical_data = geo_select_clinical_features(clinical_data, 'Adrenocortical_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
        print("Selected Clinical Data Sample:", selected_clinical_data.head())
        csv_path = './preprocessed/Adrenocortical_Cancer/trait_data/GSE143383.csv'
        selected_clinical_data.to_csv(csv_path)
        print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify the keys that store the same kind of identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [ ]:

import os 
# Create the necessary directory if it does not exist
os.makedirs('./preprocessed/Adrenocortical_Cancer/trait_data', exist_ok=True)

# Load the selected clinical data if the file exists
clinical_csv_path = './preprocessed/Adrenocortical_Cancer/trait_data/GSE143383.csv'
if not os.path.exists(clinical_csv_path):
    raise FileNotFoundError(f"Required file not found: {clinical_csv_path}")

selected_clinical_data = pd.read_csv(clinical_csv_path, index_col=0)

# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Adrenocortical_Cancer/gene_data/GSE143383.csv'
os.makedirs(os.path.dirname(gene_csv_path), exist_ok=True)
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data are severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Adrenocortical_Cancer')

# 4. Save the cohort information.
save_cohort_info('GSE143383', './preprocessed/Adrenocortical_Cancer/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Adrenocortical_Cancer/GSE143383.csv'
    unbiased_merged_data.to_csv(csv_path)
